In [38]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from planeIdentification import *
from sklearn.cluster import DBSCAN, KMeans
import shutil
import os
import time

In [8]:
def create_output_folder(directory, deleteFolder = False):
    if not(os.path.isdir(directory)):
        os.makedirs(directory)
    else:
        if(deleteFolder):
            shutil.rmtree(directory)
            os.makedirs(directory)

In [61]:
pipeline1 = ClusterPipeline([
            heightSplit(distance_threshold = 0.45),  # First clustering stage
        ])

pipeline2 = ClusterPipeline([
            DBSCAN(eps=1.5, min_samples=8),
        ])

algorithms =[{"name":"HEIGHTSPLIT", "alg": [heightSplit], "parameters": [["distance_threshold"]], "values": [[[0.3, 0.45, 0.6, 0.75, 0.9, 1.35, 1.5]]]},
             {"name":"DBSCAN", "alg": [DBSCAN], "parameters": [["eps", "min_samples"]], "values": [[[0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05, 1.2, 1.5, 1.8, 2.1], [1, 2, 4, 8, 10, 15, 20, 25, 50]]]}]

algorithms =[{"name":"DBSCAN", "alg": [DBSCAN], "parameters": [["eps", "min_samples"]], "values": [[[0.15, 1.5], [4, 10]]]},
             {"name":"HEIGHTSPLIT", "alg": [heightSplit, heightSplit], "parameters": [["distance_threshold"], ["distance_threshold"]], "values": [[[0.3, 0.45]], [[0.03, 0.045]]]},
             ]

In [74]:
import itertools


basePath = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/"
neighborhood = "Test_70_el Besòs i el Maresme"
parcelsFolder = basePath + "/Results/" + neighborhood + "/Parcels/"

for algo_dict in algorithms:
    pipeline_classes = algo_dict["alg"]
    all_parameters = algo_dict["parameters"]
    all_values = algo_dict["values"]
    name = algo_dict["name"]

    # Create all combinations of parameter values using itertools.product
    all_combinations = []
    for params, values in zip(all_parameters, all_values):
        # Create combinations for the current stage
        param_combinations = list(itertools.product(*values))
        all_combinations.append((params, param_combinations))
    
    for stage_combinations in itertools.product(*[c[1] for c in all_combinations]):
        pipeline_instances = []
        paramNames = []
        for stage_class, params, combination in zip(pipeline_classes, all_parameters, stage_combinations):
            param_dict = dict(zip(params, combination))
            pipeline_instances.append(stage_class(**param_dict))
            paramNames.append([name + "_" + str(x) for name, x in zip(params, combination)])
        paramNames = ["_".join(names) for names in paramNames] 
        
        baseOutputFolder = basePath + "/Results/" + neighborhood + "/Testing Plane ID/" + name + "_" + "__".join(paramNames) + "/"
        create_output_folder(baseOutputFolder, deleteFolder=True)
        timeList = []
        parcelsList = []
        constructionsList = []

        for parcel in os.listdir(parcelsFolder)[0:2]:                
            parcelSubfolder = parcelsFolder + parcel + "/"
            for construction in [x for x in os.listdir(parcelSubfolder)[0:1] if os.path.isdir(parcelSubfolder + x)]:
                constructionFolder = parcelSubfolder + construction

                lasPath = constructionFolder + "/Map files/" + construction + ".laz"
                lasDF = laspy.read(lasPath)

                start_time = time.time()

                pipeline = ClusterPipeline(pipeline_instances)
                pipeline.fit(lasDF.xyz)
                pipeline.getAllPlanes(lasDF.xyz)

                labels = pipeline.final_labels
                lasDF.classification = labels
                end_time = time.time()

                timeList.append(end_time - start_time)
                parcelsList.append(parcel)
                constructionsList.append(construction)

                summaryDF = pd.DataFrame({"parcel": parcelsList, "construction": constructionsList, "cluster_time": timeList})
                summaryDF.to_csv(baseOutputFolder + "Summary.csv", index=False)

                outputFolder = baseOutputFolder + "/" + parcel + "/"
                create_output_folder(outputFolder)
                lasDF.write(outputFolder + "/" + construction+".laz")

In [17]:
distancesLimits=[0.025, 0.05, 0.075, 0.10, 0.15, 0.20, 0.25, 0.3, 0.4, 0.5]
iterationsLimits=[5, 10, 20, 50, 100]

# for algorithm in algorithmLists:
for inlierThreshold, num_iterations in zip(distancesLimits, iterationsLimits):
    print(inlierThreshold, num_iterations)

0.025 5
0.05 10
0.075 20
0.1 50
0.15 100
